# **ANN Models**

### **Model Setup**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Precision, Recall

In [ ]:
# Data Loading and Preparation
df = pd.read_csv('/content/hate_speech_hindi_cleaned.csv')

In [ ]:
# F1-Score Custom Metric Function
def f1_score_metric(y_true, y_pred):
    # Cast to float for stability
    y_true = K.cast(y_true, "float32")
    y_pred = K.cast(y_pred, "float32")

    # Get Precision and Recall values
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())

    # Calculate F1-Score (Harmonic Mean)
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [ ]:
# Parameters
MAX_WORDS = 10000
MAX_LEN = 100
EMBEDDING_DIM = 100
NUM_LABELS = 5
EPOCHS = 100

In [ ]:
# Text data (X) and Labels (Y)
X = df['no_stopwords'].astype(str)
label_cols = ['defamation', 'hate', 'non-hate', 'violence', 'vulgar']
Y = df[label_cols].values

# Tokenization and Padding
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=42)


In [ ]:
# Dictionary to store results for comparison
results = {}

print("Data setup complete. Ready to train models.")

Data setup complete. Ready to train models.




*   **Tokenization**: Words are mapped to integers using a vocabulary limited to the top 10,000 words.
*   **Padding:** All sequences are uniformly set to a length of 100 by adding zeros at the end.
*   **Split:** The padded data is split into an 80% training set and a 20% testing set to ensure robust evaluation of the final model.






<br>

### **Basic ANN (Baseline)**

*Batch Size = 32*

In [ ]:
# Model Definition
model_1 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='sigmoid')
])

model_1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), AUC(name='auc'), f1_score_metric]
)

print("\n--- Training Model 1: Basic ANN ---")

# Training
model_1.fit(
    X_train, Y_train,
    epochs=EPOCHS,
    batch_size=32,
    validation_data=(X_test, Y_test),
    verbose=1
)


--- Training Model 1: Basic ANN ---
Epoch 1/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.4170 - auc: 0.6811 - f1_score_metric: 0.2337 - loss: 0.5532 - precision_11: 0.5605 - recall_11: 0.1748 - val_accuracy: 0.5121 - val_auc: 0.7855 - val_f1_score_metric: 0.5198 - val_loss: 0.4892 - val_precision_11: 0.6917 - val_recall_11: 0.4175
Epoch 2/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5722 - auc: 0.8594 - f1_score_metric: 0.6147 - loss: 0.4088 - precision_11: 0.7300 - recall_11: 0.5326 - val_accuracy: 0.5004 - val_auc: 0.7830 - val_f1_score_metric: 0.5593 - val_loss: 0.5006 - val_precision_11: 0.6574 - val_recall_11: 0.4899
Epoch 3/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6446 - auc: 0.9346 - f1_score_metric: 0.7524 - loss: 0.2976 - precision_11: 0.8033 - recall_11: 0.7095 - val_accuracy: 0.4655 - val_auc: 0.7688 - val_f1_score_metric: 0.5421 - val_loss: 0.5793 - val_precision_11: 0.6037 - val_recall_11: 0.4951
Epoch 4/100
458/458 ━━━━━━━━━━

In [ ]:
# Evaluation
print("\n--- Evaluation Model 1 ---")
loss_1, acc_1, precision_1, recall_1, auc_1, f1_1 = model_1.evaluate(X_test, Y_test, verbose=0)

# Store results
results['Model 1 (Baseline)'] = {
    'Loss': loss_1,
    'Accuracy': acc_1,
    'Precision': precision_1,
    'Recall': recall_1,
    'F1-Score': f1_1,
    'AUC': auc_1
}

# Print all metrics
print(f"Model 1 Metrics:")
print(f"  Loss: {loss_1:.4f}")
print(f"  Accuracy: {acc_1:.4f}")
print(f"  Precision: {precision_1:.4f}")
print(f"  Recall: {recall_1:.4f}")
print(f"  F1-Score: {f1_1:.4f}")
print(f"  AUC: {auc_1:.4f}")


--- Evaluation Model 1 ---
Model 1 Metrics:
  Loss: 3.7555
  Accuracy: 0.4439
  Precision: 0.5184
  Recall: 0.4842
  F1-Score: 0.4994
  AUC: 0.6799


In [ ]:
model_1.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (32, 100, 100)         │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (32, 10000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (32, 128)              │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (32, 5)                │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,842,321 (26.10 MB)

 Trainable params: 2,280,773 (8.70 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,561,548 (17.40 MB)

*Batch Size = 16*

In [ ]:
# Model Definition
model_1_2 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='sigmoid')
])

# Compile (F1-Score added to metrics list)
model_1_2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), AUC(name='auc'), f1_score_metric]
)

print("\n--- Training Model 1: Basic ANN ---")

# Training
model_1_2.fit(
    X_train, Y_train,
    epochs=EPOCHS,
    batch_size=16,
    validation_data=(X_test, Y_test),
    verbose=1
)


--- Training Model 1: Basic ANN ---
Epoch 1/100
916/916 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.4426 - auc: 0.6940 - f1_score_metric: 0.2800 - loss: 0.5462 - precision_13: 0.5670 - recall_13: 0.2193 - val_accuracy: 0.5179 - val_auc: 0.7910 - val_f1_score_metric: 0.5369 - val_loss: 0.4820 - val_precision_13: 0.6984 - val_recall_13: 0.4368
Epoch 2/100
916/916 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5766 - auc: 0.8662 - f1_score_metric: 0.6240 - loss: 0.3998 - precision_13: 0.7314 - recall_13: 0.5484 - val_accuracy: 0.5261 - val_auc: 0.7842 - val_f1_score_metric: 0.5541 - val_loss: 0.5034 - val_precision_13: 0.6476 - val_recall_13: 0.4867
Epoch 3/100
916/916 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6698 - auc: 0.9396 - f1_score_metric: 0.7648 - loss: 0.2830 - precision_13: 0.8079 - recall_13: 0.7296 - val_accuracy: 0.4696 - val_auc: 0.7688 - val_f1_score_metric: 0.5543 - val_loss: 0.5829 - val_precision_13: 0.5950 - val_recall_13: 0.5216
Epoch 4/100
916/916 ━━━━━━━━━━

In [ ]:
# Evaluation
print("\n--- Evaluation Model 1 ---")
loss_1, acc_1, precision_1, recall_1, auc_1, f1_1 = model_1_2.evaluate(X_test, Y_test, verbose=0)

# Store results
results['Model 1 (Baseline)'] = {
    'Loss': loss_1,
    'Accuracy': acc_1,
    'Precision': precision_1,
    'Recall': recall_1,
    'F1-Score': f1_1,
    'AUC': auc_1
}

# Print all metrics
print(f"Model 1 Metrics:")
print(f"  Loss: {loss_1:.4f}")
print(f"  Accuracy: {acc_1:.4f}")
print(f"  Precision: {precision_1:.4f}")
print(f"  Recall: {recall_1:.4f}")
print(f"  F1-Score: {f1_1:.4f}")
print(f"  AUC: {auc_1:.4f}")


--- Evaluation Model 1 ---
Model 1 Metrics:
  Loss: 4.7209
  Accuracy: 0.4505
  Precision: 0.5234
  Recall: 0.4754
  F1-Score: 0.4973
  AUC: 0.6800


In [ ]:
model_1_2.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (16, 100, 100)         │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (16, 10000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (16, 128)              │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (16, 5)                │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,842,321 (26.10 MB)

 Trainable params: 2,280,773 (8.70 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,561,548 (17.40 MB)

*Batch Size = 64*

In [ ]:
# Model Definition
model_1_3 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='sigmoid')
])

# Compile (F1-Score added to metrics list)
model_1_3.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall(), AUC(name='auc'), f1_score_metric]
)

print("\n--- Training Model 1: Basic ANN ---")

# Training
model_1_3.fit(
    X_train, Y_train,
    epochs=EPOCHS,
    batch_size=64,
    validation_data=(X_test, Y_test),
    verbose=1
)


--- Training Model 1: Basic ANN ---
Epoch 1/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.4308 - auc: 0.6710 - f1_score_metric: 0.1959 - loss: 0.5605 - precision_15: 0.5470 - recall_15: 0.1451 - val_accuracy: 0.5430 - val_auc: 0.7858 - val_f1_score_metric: 0.5311 - val_loss: 0.4894 - val_precision_15: 0.6808 - val_recall_15: 0.4372
Epoch 2/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5683 - auc: 0.8473 - f1_score_metric: 0.5924 - loss: 0.4255 - precision_15: 0.7286 - recall_15: 0.5005 - val_accuracy: 0.5195 - val_auc: 0.7851 - val_f1_score_metric: 0.5295 - val_loss: 0.4935 - val_precision_15: 0.6743 - val_recall_15: 0.4379
Epoch 3/100
229/229 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6513 - auc: 0.9246 - f1_score_metric: 0.7273 - loss: 0.3187 - precision_15: 0.7988 - recall_15: 0.6703 - val_accuracy: 0.4966 - val_auc: 0.7739 - val_f1_score_metric: 0.5552 - val_loss: 0.5438 - val_precision_15: 0.6233 - val_recall_15: 0.5043
Epoch 4/100
229/229 ━━━━━━━━━

In [ ]:
# Evaluation
print("\n--- Evaluation Model 1 ---")
loss_1, acc_1, precision_1, recall_1, auc_1, f1_1 = model_1_3.evaluate(X_test, Y_test, verbose=0)

# Store results
results['Model 1 (Baseline)'] = {
    'Loss': loss_1,
    'Accuracy': acc_1,
    'Precision': precision_1,
    'Recall': recall_1,
    'F1-Score': f1_1,
    'AUC': auc_1
}

# Print all metrics
print(f"Model 1 Metrics:")
print(f"  Loss: {loss_1:.4f}")
print(f"  Accuracy: {acc_1:.4f}")
print(f"  Precision: {precision_1:.4f}")
print(f"  Recall: {recall_1:.4f}")
print(f"  F1-Score: {f1_1:.4f}")
print(f"  AUC: {auc_1:.4f}")


--- Evaluation Model 1 ---
Model 1 Metrics:
  Loss: 3.2842
  Accuracy: 0.4330
  Precision: 0.5205
  Recall: 0.4777
  F1-Score: 0.4968
  AUC: 0.6845


In [ ]:
model_1_3.summary()


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ (64, 100, 100)         │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (64, 10000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (64, 128)              │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (64, 5)                │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,842,321 (26.10 MB)

 Trainable params: 2,280,773 (8.70 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,561,548 (17.40 MB)

**Model Architechture**


*   The model is a simple, feed-forward neural network for multi-label classification. The input tokens are first mapped to 100-dimensional vectors by the Embedding layer.
*   These are then flattened into a single long vector and processed by a hidden Dense layer.
*   The final output layer uses a sigmoid activation function across its 5 neurons, allowing the model to predict multiple labels simultaneously






<br>

<br>

<br>

<br>

# **ANN ( Dropout + EarlyStopping )**

In [ ]:
# Training Control
PATIENCE = 7
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True
)
MAX_EPOCHS_ES = 100

In [ ]:
# Model Definition
model_2 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM),
    Flatten(),

    Dense(256),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),

    Dense(128),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(NUM_LABELS, activation='sigmoid')
])

# Compile
model_2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc'), f1_score_metric]
)

print(f"\n--- Training Model 2  ---")


# Training
history_2 = model_2.fit(
    X_train, Y_train,
    epochs=MAX_EPOCHS_ES,
    batch_size=32,
    validation_data=(X_test, Y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluation
print("\n--- Evaluation Model 2 ---")
loss_2, acc_2, precision_2, recall_2, auc_2, f1_2 = model_2.evaluate(X_test, Y_test, verbose=0)

results['Model 2 (Best)'] = {
    'Loss': loss_2,
    'Accuracy': acc_2,
    'Precision': precision_2,
    'Recall': recall_2,
    'F1-Score': f1_2,
    'AUC': auc_2
}

# Print all metrics
print(f"Model 2 Metrics (Best Practice):")
print(f"  Loss: {loss_2:.4f}")
print(f"  Accuracy: {acc_2:.4f}")
print(f"  Precision: {precision_2:.4f}")
print(f"  Recall: {recall_2:.4f}")
print(f"  F1-Score: {f1_2:.4f}")
print(f"  AUC: {auc_2:.4f}")


--- Training Model 2  ---
Epoch 1/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.3680 - auc: 0.6470 - f1_score_metric: 0.3267 - loss: 0.5985 - precision: 0.4612 - recall: 0.2585 - val_accuracy: 0.4663 - val_auc: 0.7343 - val_f1_score_metric: 0.4613 - val_loss: 0.5347 - val_precision: 0.5362 - val_recall: 0.4081
Epoch 2/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5066 - auc: 0.8154 - f1_score_metric: 0.5536 - loss: 0.4579 - precision: 0.6753 - recall: 0.4707 - val_accuracy: 0.4802 - val_auc: 0.7683 - val_f1_score_metric: 0.5574 - val_loss: 0.5328 - val_precision: 0.6032 - val_recall: 0.5212
Epoch 3/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5877 - auc: 0.8876 - f1_score_metric: 0.6685 - loss: 0.3707 - precision: 0.7543 - recall: 0.6011 - val_accuracy: 0.4660 - val_auc: 0.7593 - val_f1_score_metric: 0.5366 - val_loss: 0.5658 - val_precision: 0.5769 - val_recall: 0.5032
Epoch 4/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6396 - 

In [ ]:
model_2.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_41 (Embedding)        │ (32, 100, 100)         │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_41 (Flatten)            │ (32, 10000)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_163 (Dense)               │ (32, 256)              │     2,560,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (32, 256)              │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_164 (Dense)               │ (32, 256)              │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (32, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_165 (Dense)               │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (32, 128)              │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_166 (Dense)               │ (32, 128)              │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (32, 128)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_167 (Dense)               │ (32, 5)                │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,031,377 (42.08 MB)

 Trainable params: 3,676,869 (14.03 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 7,353,740 (28.05 MB)

**Regularization Strategy**


*   **BatchNormalization :**  Stabilizes the learning process and accelerates convergence.
*   **Dropout (0.5 and 0.3) :** Randomly drops neurons during training, forcing the network to learn more generalized and robust features.
*   **EarlyStopping :** Monitors the validation loss and automatically halts training after 7 non-improving epochs, saving the model state with the best validation performance.





<br>

**Conclusions**


*  **Overfitting Controlled :** The Validation Loss dropped dramatically . The model is now generalizing much better.
*   **Performance Improvement :** All key metrics—Loss, F1-Score, and AUC—show significant improvement on the unseen test data.
*   **Dropout Tuning :** Different dropout rates confirm that the combination of 0.5 and 0.3 was a good choice, as lower F1 scores were observed with other combinations.







<br>

<br>

# **ANN ( With Learning Rate)**

In [ ]:
# Hyperparameters
LR = 0.0005
PATIENCE = 7
MAX_EPOCHS_ES = 100

# Training Control
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=PATIENCE,
    restore_best_weights=True
)

In [ ]:
# Model Definition
model_3 = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM),
    Flatten(),

    Dense(256),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),

    Dense(128),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(NUM_LABELS, activation='sigmoid')
])

# Compile
custom_adam = Adam(learning_rate=LR)

model_3.compile(
    optimizer=custom_adam,
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc'), f1_score_metric]
)

print(f"\n--- Training Model 3 (Custom LR: {LR}) ---")

# Training
history_3 = model_3.fit(
    X_train, Y_train,
    epochs=MAX_EPOCHS_ES,
    batch_size=32,
    validation_data=(X_test, Y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluation
print("\n--- Evaluation Model 3 ---")
loss_3, acc_3, precision_3, recall_3, auc_3, f1_3 = model_3.evaluate(X_test, Y_test, verbose=0)

# Store results
results['Model 3 (Custom LR)'] = {
    'Loss': loss_3,
    'Accuracy': acc_3,
    'Precision': precision_3,
    'Recall': recall_3,
    'F1-Score': f1_3,
    'AUC': auc_3
}

# Print all metrics
print(f"Model 3 Metrics (Custom LR):")
print(f"  Loss: {loss_3:.4f}")
print(f"  Accuracy: {acc_3:.4f}")
print(f"  Precision: {precision_3:.4f}")
print(f"  Recall: {recall_3:.4f}")
print(f"  F1-Score: {f1_3:.4f}")
print(f"  AUC: {auc_3:.4f}")


--- Training Model 3 (Custom LR: 0.0005) ---
Epoch 1/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 26s 48ms/step - accuracy: 0.3333 - auc: 0.6144 - f1_score_metric: 0.3095 - loss: 0.6241 - precision: 0.4114 - recall: 0.2633 - val_accuracy: 0.1168 - val_auc: 0.5210 - val_f1_score_metric: 0.3007 - val_loss: 0.6722 - val_precision: 0.2858 - val_recall: 0.3215
Epoch 2/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.4992 - auc: 0.7943 - f1_score_metric: 0.5333 - loss: 0.4797 - precision: 0.6618 - recall: 0.4480 - val_accuracy: 0.4180 - val_auc: 0.7661 - val_f1_score_metric: 0.5252 - val_loss: 0.5074 - val_precision: 0.6694 - val_recall: 0.4347
Epoch 3/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.5683 - auc: 0.8709 - f1_score_metric: 0.6421 - loss: 0.3927 - precision: 0.7301 - recall: 0.5745 - val_accuracy: 0.4775 - val_auc: 0.7220 - val_f1_score_metric: 0.5017 - val_loss: 0.6048 - val_precision: 0.5240 - val_recall: 0.4865
Epoch 4/100
458/458 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/s



*   The resulting F1−Score of 0.5252 is lower than Model 2's 0.5574.

